In [3]:
from datasets import load_dataset


dataset = load_dataset("DataForGood/climateguard", split="test")

In [5]:
example = dataset[0]
example

{'label_studio_id': 2160,
 'id': 'f5d30bfcb029eb6de736353e72d9bdb987e1a70b3849eb3f3cdb536512862567',
 'day': 7,
 'year': 2025,
 'month': 3,
 'start': '2025-03-07T11:24:00+00:00',
 'channel': 'rmc',
 'plaintext': "alors on vous entend super mal donc je vous disais vous passez tous les jours à côté de changé la bad not les e allons aujourd' hui un niveau infrastructure si on ne veut pas de grosses grosses entreprises qui partiront comme des entreprises pharmaceutiques convoqué dans le cadre qui abordent aujourd' hui dès huit mille salariés pas de bêtise ça rit ça brise la ligne malheureusement allez sont allez mauvais un on enjeu est un économique enjeu très économique très fort et on peut parvenir on ne peut pas revenir en arrière on est désolés favorisant la ligne était trop mauvais on intervenait avec louis alors de senlis en haute-garonne il est conducteur d' engins bonjour louis un beau jour toute l' équipe conducteurs d' engins vous étiez sur euh sur le chantier d' autoroute justem

In [1]:
from mlx_lm import load, generate

model_lora, tokenizer_lora = load(
    path_or_hf_repo="microsoft/Phi-3-mini-4k-instruct", 
    adapter_path="adapters"
)

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

In [8]:
messages = [
    {"role": "user", "content": example["plaintext"]}
]

prompt = tokenizer_lora.apply_chat_template(messages, add_generation_prompt=True)

response = generate(
    model_lora,
    tokenizer_lora,
    prompt=prompt,
    max_tokens=1000,
    verbose=False
)


In [ ]:
import re
import json


def parse_json(text):
    pattern = r'\{.*?\}'  # Matches JSON-like objects

    match = re.search(pattern, text, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            json_obj = json.loads(json_str)  # Properly parse the JSON
            return json_obj
        except json.JSONDecodeError:
            print("Cannot decode JSON")
            return None
    print("No match found")


In [ ]:
from tqdm.notebook import tqdm
predictions = []
labels = []

for example in tqdm(dataset):
    messages = [
        {"role": "user", "content": example["plaintext"]}
    ]

    prompt = tokenizer_lora.apply_chat_template(messages, add_generation_prompt=True)

    response = generate(
        model_lora,
        tokenizer_lora,
        prompt=prompt,
        max_tokens=1024,
        verbose=False
    )
    predictions.append(parse_json(response)["score"] if parse_json(response) else 0)
    labels.append(int(example["misinformation"]))

  0%|          | 0/119 [00:00<?, ?it/s]

TypeError: 'NoneType' object is not subscriptable

In [25]:
labels = [int(label) * 10 for label in labels]

In [27]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions))

              precision    recall  f1-score   support

           0       0.68      0.78      0.73        67
          10       0.48      0.37      0.42        38

    accuracy                           0.63       105
   macro avg       0.58      0.57      0.57       105
weighted avg       0.61      0.63      0.62       105



In [26]:
labels

[0,
 10,
 0,
 0,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 10,
 0,
 0,
 0,
 0,
 10,
 10,
 10,
 0,
 0,
 10,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 10,
 0,
 10,
 0,
 0,
 10,
 10,
 10,
 10,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 10,
 10,
 10,
 10,
 0,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 10,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 10,
 0,
 0,
 0,
 10,
 0,
 0,
 0,
 10,
 10]